# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [17]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://books.toscrape.com/"
response = requests.get(url)


# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [18]:

soup = BeautifulSoup(response.content, "html.parser")
categories = soup.select('.nav-list a')

cat_url = []

for category in categories:
    cat_url.append(category["href"])


# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [19]:
def get_book_urls(category_url):
    response = requests.get(url + category_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    book_links = soup.select('h3 a')
    book_urls = [url + 'catalogue/' + link.get('href') for link in book_links]
    return book_urls


# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [20]:


def book_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    title = soup.h1.text
    price = soup.select_one('.price_color').text
    availability = soup.select_one('.instock.availability').text.strip()
    rating = len(soup.select('p.star-rating')[0].get('class')[1])
    description = soup.select_one('#product_description + p').text if soup.select_one('#product_description + p') else 'No description available'
    upc = soup.select_one('th:contains("UPC") + td').text

    dic = {
        "Title": title,
        "Price": price,
        "Availability": availability,
        "Rating": rating,
        "Description": description,
        "UPC": upc
    }
    
    return dic



# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [21]:
from tqdm import tqdm

books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }

for category in tqdm(cat_url):
    book_url = get_book_urls(category)

    for book_url in book_urls:
        book_info = book_details(book_url)
        books_dict["Title"].append(book_info["Title"])
        books_dict["Price"].append(book_info["Price"])
        books_dict["Availability"].append(book_info["Availability"])
        books_dict["Rating"].append(book_info["Rating"])
        books_dict["Description"].append(book_info["Description"])
        books_dict["UPC"].append(book_info["UPC"])
        books_dict["Category"].append(category_name)

# Creating DataFrame from the dictionary
df = pd.DataFrame(books_dict)

# Display the DataFrame
print(df)

  0%|          | 0/51 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'text'